# DL1 Assignment2 - Q1.1

This is a small help from us to save you some coding. This notebook is **not** graded, you are free to edit it.

Further advise:
1. Start with File/Save a copy in Drive
2. Set GPU usage under Runtime/Change runtime type/Hardware accelerator.

In [1]:
!pip install timm kaleido

In [2]:
import torch
from torch import nn
import timm
from torchvision import models
from matplotlib import pyplot as plt
from typing import Callable
from prettytable import PrettyTable
import numpy as np
import pandas as pd
from tqdm import autonotebook as tqdm
import plotly.express as px

%matplotlib inline

In [11]:
def vit_s_8():
    """ViT-S/8 is not a default torchvision model, so we provide it by timm"""
    # Accuracy approximation comes from
    # https://openreview.net/pdf?id=LtKcMgGOeLt
    # and DINO
    # https://arxiv.org/abs/2104.14294
    return timm.create_model('vit_small_patch8_224')

# Model definitions
# Optional Q: These are uncalled functions. What do you think would happen
# if we called all of them once? Why didn't we do that?
model_defs = [
    vit_s_8,
    models.vit_b_32,
    models.vgg11,
    models.vgg11_bn,
    models.resnet18,
    models.densenet121,
    models.mobilenet_v3_small,
]

# Accuracies per model
model_accs = {
    'vit_s_8': 80., # Approximated
    'vit_b_32' : 75.912,
    'vgg11' : 69.02,
    'vgg11_bn' : 70.37,
    'resnet18' : 69.758,
    'densenet121' : 74.434,
    'mobilenet_v3_small' : 67.668,
}


def measure_runtime_per_forward(model:nn.Module, no_grad:bool, batch_size:int=8):
    """Measures the time for a single pass in milliseconds"""

    # Generate fake RGB input (224x224)
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inp = torch.rand(batch_size, 3, 224, 224,
                     #dtype=torch.cuda.FloatTensor
                     )
    inp = inp.to(device)
    #######################
    # END OF YOUR CODE    #
    #######################

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()

    # Run the model
    #######################
    # PUT YOUR CODE HERE  #
    #######################
    model.requires_grad_(no_grad)
    model.eval()
    outputs = model(inp)
    #######################
    # END OF YOUR CODE    #
    #######################

    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)


def count_model_parameters(model, print_debug=False):
    if print_debug:
        table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        if print_debug:
            table.add_row([name, params])
        total_params += params
    if print_debug:
        print(table)
        print(f"Total Trainable Params: {total_params}")
    return total_params


def filter_outliers_iqr(data, iqr_cutoff_multiplier=1.5):
    """ Filter outliers based on the Inter-Quartile range """
    q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
    iqr = q75 - q25
    cut_off = iqr * iqr_cutoff_multiplier
    lower, upper = q25 - cut_off, q75 + cut_off
    outliers = [x for x in data if x < lower or x > upper]
    clean_data = [x for x in data if x >= lower and x <= upper]
    return clean_data, outliers

def evaluate_model(model_def:Callable, no_grad:bool, batch_size:int=8, n_test_batches:int=20):

    # Retreive initial memory allocation
    initial_vram = torch.cuda.memory_allocated()

    # Define model
    model = model_def().cuda().eval()
    # Access name as: model.__name__

    # Parameters that need to be filled
    n_params = None
    times, vrams = [], []
    mean_time = None
    mean_vram = None

    #######################
    # PUT YOUR CODE HERE  #
    #######################

    # Step 1: Calculate the number of **trainable** parameters
    n_params = count_model_parameters(model)
    # Step 2: Warm up with a few passes
    # Step 3: Run N forward passes and save the runtime +
    #         the vram allocated by the model
    for i in range(n_test_batches):
        runtime = measure_runtime_per_forward(model, no_grad, batch_size)
        vram = torch.cuda.memory_allocated()
        times.append(runtime)
        vrams.append(vram)
    # Step 4: Take the mean, preferably with dropping possible outliers
    inlier_times, outlier_times = filter_outliers_iqr(times, iqr_cutoff_multiplier=1.5)
    print(f"{len(inlier_times)} times kept, {len(outlier_times)} removed. {inlier_times}, {outlier_times}")
    mean_time = np.mean(inlier_times) / batch_size
    vrams = np.array(vrams) - initial_vram
    mean_vram = vrams.mean()
    mean_vram /= (1024 ** 2) # convert to mb


    #######################
    # END OF YOUR CODE    #
    #######################

    # Clean up space for the model
    del model
    torch.cuda.empty_cache()

    return mean_time, mean_vram, n_params


In [12]:
#######################
# PUT YOUR CODE HERE  #
#######################

# Make your plots here with matplotlib
#
# Example usage of the above functions:
def evaluate_models(batch_size:int=8, n_test_batches:int=20):
    results = []
    for no_grad in [True, False]:
        for model_def in tqdm.tqdm(model_defs, desc=f"Batch size {batch_size}, no_grad {no_grad}", leave=False):
            name = model_def.__name__
            time, vram, n_params = evaluate_model(model_def, no_grad=no_grad, n_test_batches=n_test_batches)
            result = {
                "name": name,
                "top1_acc": model_accs[name],
                "batch_size": batch_size,
                "no_grad": no_grad,
                "time": time,
                "vram": vram,
                "n_params": n_params
            }
            results.append(result)
            #print(name, time, vram, n_params)
    results_df = pd.DataFrame(results)
    return results_df

results_df_batch8 = evaluate_models(batch_size=8, n_test_batches=20)
results_df_batch64 = evaluate_models(batch_size=8, n_test_batches=20)
#######################
# END OF YOUR CODE    #
#######################

Batch size 8, no_grad True:   0%|          | 0/7 [00:00<?, ?it/s]

18 times kept, 2 removed. [104.64611053466797, 109.32870483398438, 104.55401611328125, 106.16012573242188, 106.98287963867188, 106.64348602294922, 104.5285415649414, 106.35692596435547, 108.078369140625, 104.44780731201172, 105.45097351074219, 108.20054626464844, 105.91747283935547, 105.67225646972656, 105.59410858154297, 103.80147552490234, 104.79811096191406, 103.882080078125], [184.23516845703125, 115.5340805053711]
20 times kept, 0 removed. [18.827775955200195, 25.017248153686523, 25.189151763916016, 25.05299186706543, 24.987327575683594, 25.015296936035156, 22.01692771911621, 20.08742332458496, 19.881216049194336, 21.62371253967285, 20.9401912689209, 19.92255973815918, 20.20684814453125, 21.776384353637695, 21.040000915527344, 20.079328536987305, 20.09494400024414, 22.010400772094727, 20.809280395507812, 19.911712646484375], []
17 times kept, 3 removed. [31.28009605407715, 29.26473617553711, 29.234495162963867, 28.854496002197266, 28.845792770385742, 25.196544647216797, 24.7752323

Batch size 8, no_grad False:   0%|          | 0/7 [00:00<?, ?it/s]

18 times kept, 2 removed. [107.05728149414062, 105.6822738647461, 106.61984252929688, 106.40713500976562, 105.91193389892578, 106.14313507080078, 106.50835418701172, 106.92950439453125, 104.83865356445312, 106.38796997070312, 106.03218841552734, 105.97138977050781, 105.41548919677734, 105.5362548828125, 105.43462371826172, 106.01827239990234, 104.41292572021484, 106.17314910888672], [100.96873474121094, 108.39904022216797]
16 times kept, 4 removed. [30.410303115844727, 24.365663528442383, 24.36137580871582, 23.885343551635742, 23.824384689331055, 23.073728561401367, 19.154464721679688, 20.295551300048828, 19.81443214416504, 18.59872055053711, 21.341503143310547, 19.796127319335938, 19.187135696411133, 21.33283233642578, 19.472320556640625, 19.6595516204834], [39.81449508666992, 39.657535552978516, 39.59743881225586, 39.56550216674805]
18 times kept, 2 removed. [35.076576232910156, 30.527135848999023, 30.510143280029297, 30.504671096801758, 30.51817512512207, 29.604448318481445, 25.4433

Batch size 8, no_grad True:   0%|          | 0/7 [00:00<?, ?it/s]

18 times kept, 2 removed. [108.83491516113281, 107.36406707763672, 107.05974578857422, 106.43567657470703, 108.65283203125, 107.57158660888672, 107.62406158447266, 108.83507537841797, 108.95836639404297, 105.90496063232422, 109.1816635131836, 107.57526397705078, 109.20531463623047, 107.90153503417969, 108.3253402709961, 109.43292999267578, 107.69004821777344, 109.61478424072266], [100.95683288574219, 118.40742492675781]
20 times kept, 0 removed. [18.798784255981445, 20.292512893676758, 19.83407974243164, 20.736032485961914, 22.95155143737793, 21.31817626953125, 20.725696563720703, 20.20307159423828, 23.312063217163086, 24.12224006652832, 21.113727569580078, 20.534080505371094, 20.0578556060791, 20.854816436767578, 22.83776092529297, 21.81270408630371, 19.90598487854004, 20.770751953125, 22.759872436523438, 22.023296356201172], []
16 times kept, 4 removed. [31.391008377075195, 29.30361557006836, 29.285791397094727, 28.921951293945312, 28.869504928588867, 24.751903533935547, 25.671199798

Batch size 8, no_grad False:   0%|          | 0/7 [00:00<?, ?it/s]

19 times kept, 1 removed. [108.09967803955078, 107.18962860107422, 107.65225219726562, 107.97830200195312, 107.53206634521484, 106.6401596069336, 108.46406555175781, 108.38694763183594, 108.34393310546875, 104.71459197998047, 106.80524444580078, 105.11100769042969, 108.4095687866211, 107.64886474609375, 106.30274963378906, 106.91871643066406, 107.14790344238281, 105.34220886230469, 107.07539367675781], [104.08860778808594]
16 times kept, 4 removed. [27.117984771728516, 19.837535858154297, 19.089696884155273, 20.98192024230957, 19.830080032348633, 19.030048370361328, 20.61814308166504, 20.900991439819336, 19.329952239990234, 19.798240661621094, 21.60393524169922, 20.178911209106445, 18.867647171020508, 21.145856857299805, 20.22377586364746, 18.944704055786133], [29.902559280395508, 29.85856056213379, 29.792448043823242, 29.760576248168945]
16 times kept, 4 removed. [27.166240692138672, 26.723743438720703, 26.56070327758789, 26.607872009277344, 26.73311996459961, 27.2891845703125, 25.487

In [42]:
labels={
        "time": "Elapsed time per sample (ms)",
        "n_params": "Number of model parameters",
        "top1_acc": "Top1 accuracy of model on ImageNet (%)",
        "name": "Model name",
        "vram": "Amount of VRAM used during forward pass (MB)"
    }
fig = px.scatter(
    results_df_batch8.query("no_grad == True").sort_values("top1_acc"),
    x="top1_acc",
    y="time",
    text="name",
    trendline="ols",
    labels=labels,
    title="Top1 accuracy of model on ImageNet as a function of inference speed",
    width=800,
    height=500
)

def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation
    positions = ['top right', 'bottom right', 'bottom left', 'top left']
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(results_df['n_params']))
fig.show()
fig.write_image('top1_per_runtime.png')

In [44]:

fig = px.scatter(
    results_df_batch8.sort_values("n_params"),
    x="n_params",
    y="time",
    text="name",
    labels=labels,
    title="Mean forward pass time per sample as a function of model size, without and with gradient calculation",
    facet_col="no_grad",
    width=1200,
    height=500
)

fig.update_traces(textposition=improve_text_position(results_df['n_params']))
fig.show()
fig.write_image('runtime_per_model_size.png')

In [45]:
fig = px.bar(
    results_df_batch64,
    x="name",
    y="vram",
    color="no_grad",
    barmode="group",
    labels=labels,
    title="VRAM usage during forward pass per model, without and with gradient calculation",
    width=1200,
    height=500
)

def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation
    positions = ['top left', 'bottom right', 'bottom left', 'top right']
    return [positions[i % len(positions)] for i in range(len(x))]

# fig.update_traces(textposition=improve_text_position(results_df['n_params']))
fig.show()
fig.write_image('vram_per_model_size.png')